Assignment - 

Use DVC or MLFLOW to convert this into a 
1. Create and API which will take the first input and it'll generate the next sentences
or 
2. Create and UI for this app

Submit it before Friday.

# Text generation with an RNN

In [1]:
import os
ROOT = "/content/drive/MyDrive/001_SUNNY_BHAVEEN_CHANDRA/001_My_Classes/FSDS_Feb_Batch/ALL_NOTES/May-21-22"
os.chdir(ROOT)

In [2]:
import tensorflow as tf
import numpy as np
import time

In [3]:
os.makedirs("data", exist_ok=True)

In [4]:
!curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt > "./data/shakespeare.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  6641k      0 --:--:-- --:--:-- --:--:-- 6682k


In [5]:
# samethinng can be done using config.yaml
class Config:
  path_to_file = os.path.join("data", "shakespeare.txt")
  seq_length = 100

  Batch_size = 64
  Buffer_size = 10000 # 

  embedding_dim = 256

  rnn_units = 1024

  EPOCHS = 30

  checkpoint_dir = "./training_ckpt"

In [6]:
Config.EPOCHS

30

In [7]:
text = open(Config.path_to_file, "rb").read().decode(encoding='utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [8]:
with open(Config.path_to_file) as f:
  t = f.read()

In [9]:
t[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [10]:
vocab = sorted(set(text))
len(vocab)

65

In [11]:
char2idx = {uniChar: idx for idx, uniChar in enumerate(vocab)}
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [12]:
text_as_int = np.array([char2idx[c] for c in text])

text_as_int # all text is now represented as integer

array([18, 47, 56, ..., 45,  8,  0])

In [13]:
text[:13], text_as_int[:13]

('First Citizen', array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]))

In [14]:
len(text)

1115394

In [15]:
examples_per_epoch = len(text)//(Config.seq_length + 1)
examples_per_epoch

11043

In [16]:
idx2char_DICT = {val: key for key, val in char2idx.items()}

In [17]:
idx2char = np.array(vocab)
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [18]:
idx2char[0], idx2char_DICT[0] 

('\n', '\n')

In [19]:
print("Hi\nThere")

Hi
There


In [20]:
print(r"Hi\nThere")

Hi\nThere


In [21]:
print(repr("Hi\nThere"))

'Hi\nThere'


In [22]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(13):
  print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i
z
e
n


In [23]:
sequences = char_dataset.batch(Config.seq_length + 1, drop_remainder=True)

for item in sequences.take(10):
  to_print = repr("".join([idx2char[c] for c in item.numpy()]))
  print(to_print)
  print(len(to_print))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
110
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
109
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
109
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
107
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
109
'zens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but th'
105
'e superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we a'
105
're too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particula'
105
'rise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we 

In [25]:
def split_input_target(chunk):
  input_text = chunk[:-1] # First Citize
  target_text = chunk[1:] # irst Citizen
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [26]:
for input_example, target_example in dataset.take(1):
  print("input_data:\n")
  print(repr("".join([idx2char[i] for i in input_example.numpy()])))
  print("\n\ntarget_data:\n")
  print(repr("".join([idx2char[t] for t in target_example.numpy()])))

input_data:

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'


target_data:

'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [27]:
dataset = dataset.shuffle(Config.Buffer_size).batch(Config.Batch_size, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [29]:
Config.vocab_size = len(vocab)
Config.vocab_size

65

In [30]:
def build_model(
    vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
          tf.keras.layers.GRU(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(vocab_size)
  ])
  return model


In [31]:
model = build_model(
    vocab_size = Config.vocab_size,
    embedding_dim = Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = Config.Batch_size
)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [33]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [34]:
model.compile(optimizer='adam', loss=loss)

In [35]:
checkpoint_prefix = os.path.join(Config.checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [36]:
history = model.fit(dataset, epochs=Config.EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 17s 56ms/step - loss: 2.6506
Epoch 2/30
172/172 [==============================] - 11s 57ms/step - loss: 1.9448
Epoch 3/30
172/172 [==============================] - 11s 58ms/step - loss: 1.6816
Epoch 4/30
172/172 [==============================] - 11s 58ms/step - loss: 1.5366
Epoch 5/30
172/172 [==============================] - 11s 58ms/step - loss: 1.4500
Epoch 6/30
172/172 [==============================] - 11s 59ms/step - loss: 1.3912
Epoch 7/30
172/172 [==============================] - 11s 58ms/step - loss: 1.3464
Epoch 8/30
172/172 [==============================] - 11s 58ms/step - loss: 1.3077
Epoch 9/30
172/172 [==============================] - 11s 58ms/step - loss: 1.2734
Epoch 10/30
172/172 [==============================] - 11s 59ms/step - loss: 1.2393
Epoch 11/30
172/172 [==============================] - 11s 59ms/step - loss: 1.2070
Epoch 12/30
172/172 [==============================] - 11s 58ms/step - loss: 1.1750
E

In [37]:
(len(text)/Config.Batch_size)/(Config.seq_length + 1)

172.55476485148515

In [38]:
# Restoring checkoint - 

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/ckpt_30'

In [39]:
model_from_ckpt = build_model(
    vocab_size = Config.vocab_size,
    embedding_dim = Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = 1
)

In [ ]:
model_from_ckpt.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_2 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_2 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [40]:
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape([1, None]))

In [41]:
model_from_ckpt.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


# prediction

In [42]:
# define a 
def generate_text(model, start_string, no_of_chars_to_gen=1000):
  # convert the input text to nos.
  input_val = [char2idx[s] for s in start_string] # text converted to int
  input_val = tf.expand_dims(input_val, 0) # [] ->> [1, ]

  text_generated = list()

  temperature = 1.0


  model.reset_states() # Resetting the previous states if any while predictions.
  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)

    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    # print(predictions)
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    # print(predicted_id)
    
    input_val = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return start_string + "".join(text_generated)

In [43]:
result = generate_text(model=model_from_ckpt, start_string="SUNNY")
print(result)

SUNNY:
The sun unwash'd by him, my lords, do give leave amazed me, had you
have we t.

SLY:
Then I say, I'll bring thee to the ground
To counterfeit the heir of Rome, he must beseem
My strive required in thou rascals.

TRANIO:
Faith, I call your power.
And take your blood to see his absence; he
More holy looking to be doned?

LUCIO:
If it be so; for whence should you go
The unstain'd wife, and the game your brother come,
Where he did close by Edward's gown, by time to touch a quarrel mildle fortune;
To be her my brother, were I offended me, then All-Souls' day to make you gatiently to thy lord,
For which is my wife's cloud stone but England's queen.
Food queen, your heads!

MERCUTIO:
Nor, you come to the Earl Grey,
For dishonour'd with this forlorn-time is strange.
Thieves for this once, if they bear you then.

HASTINGS:
I got them in a monarch'd toar,
Shall we cenving to the ben; o'er his own
Shore five but this gentleman by no staff, sir, but, and by the ears,
Volsces, minister, foun

In [44]:
tf.math.log([[0.5, 0.5]])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.6931472, -0.6931472]], dtype=float32)>

In [45]:

for _ in range(15):
  predictions = [[1000., 1.,2.,3.,4., 55., 56., 100., 101., 200., 1001]]
  samples = tf.random.categorical(predictions, 1)[-1, 0]
  print(samples.numpy())
  # print(values[samples.numpy()])

0
0
10
10
10
10
10
10
10
10
0
10
10
10
0
